# Внутриигровые покупки
_____________

Автор:  Алена Свиридова (GitHub - AlenaCarp)

_____________
## Введение

### Цели и задачи проекта

Изучить влияние характеристик игроков и их игровых персонажей на покупку внутриигровой валюты «райские лепестки», а также оценить активность игроков при совершении внутриигровых покупок.   
Анализ товара в игровом магазине.  
Выявить популярный товар.

### Содержание проекта

   1. [Описание данных](#1-bullet)
   2. [Подключение к БД](#2-bullet)
   3. [Знакомство с данными](#3-bullet)
   4. [Анализ данных](#4-bullet)
   5. [Общие выводы и рекомендации](#5-bullet)

<a class="anchor" id="1-bullet"></a>
## Описание данных

Схема fantasy содержит 7 таблиц:  

    игроки - users;
    внутриигровые покупки - events;
    класса персонажа — таблица classes;  
    легендарного умения — таблица skills;  
    расы персонажа — таблица race;  
    эпического предмета — таблица items;  
    страны регистрации игрока — таблица country;  

![Example Image](схема.png)

_______
<a class="anchor" id="2-bullet"></a>
## Подключение к БД

Используем подключение через SQLAlchemy, потому что Pandas официально поддерживает только три типа соединений для pd.read_sql(), и прямое подключение через psycopg2 не входит в их число.

In [2]:
# Библиотеки
from sqlalchemy import create_engine
import pandas as pd

In [3]:
# Подключение к БД

# Параметры подключения
username = ''
password = ''
host = ''
port = ''
database = ''

# Формируем строку подключения
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

______________
<a class="anchor" id="3-bullet"></a>
## Знакомство с данными

Выведите названия всех таблиц схемы **fantasy**.

In [4]:
query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'fantasy'
        AND table_type = 'BASE TABLE';
"""
pd.read_sql(query, con=engine)

,table_name
0,classes
1,country
2,events
3,items
4,race
5,skills
6,users


Данные в таблице **users**. Получим информацию о названии полей таблицы и типе данных в них, а также о первичных и внешних ключах.

In [5]:
query = """
SELECT
    c.table_schema,
    c.table_name,
    c.column_name,
    c.data_type,
    kcu.constraint_name
FROM information_schema.columns AS c
LEFT JOIN information_schema.key_column_usage AS kcu
    ON  c.table_schema = kcu.table_schema
    AND c.table_name  = kcu.table_name
    AND c.column_name = kcu.column_name
LEFT JOIN information_schema.table_constraints AS tc
    ON  kcu.table_schema = tc.table_schema
    AND kcu.table_name   = tc.table_name
    AND kcu.constraint_name = tc.constraint_name
    AND tc.constraint_type IN ('PRIMARY KEY','FOREIGN KEY')
WHERE c.table_schema = 'fantasy'
  AND c.table_name   = 'users'
ORDER BY c.ordinal_position;
"""
pd.read_sql(query, con=engine)

,table_schema,table_name,column_name,data_type,constraint_name
0,fantasy,users,id,character varying,users_pkey
1,fantasy,users,tech_nickname,character varying,None
2,fantasy,users,class_id,character varying,users_class_id_fkey
3,fantasy,users,ch_id,character varying,users_ch_id_fkey
4,fantasy,users,birthdate,character varying,None
5,fantasy,users,pers_gender,character varying,None
6,fantasy,users,registration_dt,character varying,None
7,fantasy,users,server,character varying,None
8,fantasy,users,race_id,character varying,users_race_id_fkey
9,fantasy,users,payer,integer,None


5 строк из табоицы users

In [6]:
query = """
SELECT *,COUNT(*)OVER() AS row_count
FROM fantasy.users
LIMIT 5
"""
pd.read_sql(query, con=engine)

,id,tech_nickname,class_id,ch_id,birthdate,pers_gender,registration_dt,server,race_id,payer,loc_id,row_count
0,00-0037846,DivineBarbarian4154,9RD,JJR2,6/4/1994,Male,1/20/2005,server_1,B1,0,US,22214
1,00-0041533,BoldInvoker7693,Z3Q,HQ9N,6/29/1987,Male,4/8/2022,server_1,R2,0,US,22214
2,00-0045747,NobleAlchemist7633,382,IXBW,7/29/1992,Male,10/12/2013,server_1,K3,0,US,22214
3,00-0055274,SteadfastArcher8318,ZD0,QSUB,9/14/1985,Female,4/10/2008,server_1,R2,0,US,22214
4,00-0076100,RadiantProphet353,YC8,HQ9N,4/11/1997,Female,9/29/2013,server_2,K4,1,US,22214


Проверка пропусков в **users**

In [7]:
query = """
SELECT COUNT(*)
FROM fantasy.users
WHERE class_id IS NULL OR ch_id IS NULL OR pers_gender IS NULL OR server IS NULL OR race_id IS NULL OR payer IS NULL OR loc_id IS NULL
"""
pd.read_sql(query, con=engine)

,count
0,0


Найдем количество игроков на каждом сервере.

In [8]:
query = """
SELECT server, COUNT(*)
FROM fantasy.users
GROUP BY server
"""
pd.read_sql(query, con=engine)

,server,count
0,server_2,5499
1,server_1,16715


Выведим названия всех полей, их тип данных и информацию о ключевых полях таблицы **events**.

In [9]:
query = """
-- Выводим названия полей, их тип данных и метку о ключевом поле таблицы events
SELECT c.table_schema,
       c.table_name,
       c.column_name,
       c.data_type,
       k.constraint_name
FROM information_schema.columns AS c 
-- Присоединяем данные с ограничениями полей
LEFT JOIN information_schema.key_column_usage AS k 
    USING(table_name, column_name, table_schema)
-- Фильтруем результат по названию схемы и таблицы
WHERE c.table_schema = 'fantasy'
  AND c.table_name   = 'events'
ORDER BY c.table_name;
"""
pd.read_sql(query, con=engine)

,table_schema,table_name,column_name,data_type,constraint_name
0,fantasy,events,transaction_id,character varying,events_pkey
1,fantasy,events,id,character varying,events_id_fkey
2,fantasy,events,date,character varying,None
3,fantasy,events,time,character varying,None
4,fantasy,events,item_code,integer,events_item_code_fkey
5,fantasy,events,amount,real,None
6,fantasy,events,seller_id,character varying,None


5 строк таблицы **events**

In [10]:
query = """
SELECT *,COUNT(*)OVER() AS row_count
FROM fantasy.events
LIMIT 5
"""
pd.read_sql(query, con=engine)

,transaction_id,id,date,time,item_code,amount,seller_id,row_count
0,2129235853,37-5938126,2021-01-03,16:31:49,6010,21.41,220381,1307678
1,2129237617,37-5938126,2021-01-03,16:49:00,6010,64.98,54680,1307678
2,2129239381,37-5938126,2021-01-03,21:05:29,6010,50.68,888909,1307678
3,2129241145,37-5938126,2021-01-03,22:03:02,6010,46.49,888902,1307678
4,2129242909,37-5938126,2021-01-03,22:04:26,6010,18.72,888905,1307678


Проверяем на пропуски **events**

In [11]:
query = """
SELECT COUNT(*)
FROM fantasy.events
WHERE date IS NULL OR time IS NULL OR amount IS NULL OR seller_id IS NULL
"""
pd.read_sql(query, con=engine)

,count
0,508186


В 508186 строках из 1307678 встречаются пропуски хотя бы в одном из полей.

Проверим строки с пропусками, посмотрим какие столбцы содержат их.

In [12]:
query = """
-- Считаем количество строк с данными в каждом поле
SELECT 
SUM(CASE WHEN date IS NOT NULL THEN 1 ELSE 0 END) AS data_count,
SUM(CASE WHEN time IS NOT NULL THEN 1 ELSE 0 END) AS data_time,
SUM(CASE WHEN amount IS NOT NULL THEN 1 ELSE 0 END) AS data_amount,
SUM(CASE WHEN seller_id IS NOT NULL THEN 1 ELSE 0 END) AS data_seller_id
FROM fantasy.events
WHERE date IS NULL
  OR time IS NULL
  OR amount IS NULL
  OR seller_id IS NULL;
"""
pd.read_sql(query, con=engine)

,data_count,data_time,data_amount,data_seller_id
0,508186,508186,508186,0


Нет информации только о продавцах. Это может указывать на то что покупка совершалась в магазине.

______
<a class="anchor" id="4-bullet"></a>
## Анализ данных

### Исследование платящих игроков

Доля платящих пользователей по всем данным:

In [13]:
query = """
SELECT COUNT(id) AS count_players, 
	SUM(payer) AS count_payers, 
	AVG(payer::NUMERIC)::NUMERIC(4,3) AS payer_share
FROM fantasy.users;
"""
pd.read_sql(query, con=engine)

,count_players,count_payers,payer_share
0,22214,3929,0.177


Доля платящих пользователей в разрезе расы персонажа:

In [14]:
query = """
SELECT race, 
	SUM(payer) AS count_payers, 
	COUNT(id) AS count_players, 
	(SUM(payer::NUMERIC)/COUNT(id))::NUMERIC(4,3) AS payer_share
FROM fantasy.users
LEFT JOIN fantasy.race USING(race_id)
GROUP BY race
ORDER BY payer_share DESC;
"""
pd.read_sql(query, con=engine)

,race,count_payers,count_players,payer_share
0,Demon,238,1229,0.194
1,Hobbit,659,3648,0.181
2,Orc,636,3619,0.176
3,Human,1114,6328,0.176
4,Northman,626,3562,0.176
5,Angel,229,1327,0.173
6,Elf,427,2501,0.171


Вывод:  
17.7% игроков совершают платежи.  
Разброс доли платящих игроков между расами минимален: от 17.1% до 19.4%  
У самой платящей расы самая низкая численность - 1229. Кол-во платящих - 238.  
За счет численности, самое большое кол-во платящих у Human - 1114, что превышает кол-во платящих у демонов в 4,7 раза.  
Human Orc Hobbit формируют основной поток платящих пользователей за счет численности.  



### Исследование внутриигровых покупок

Основные статистические показатели по полю amount-стоимость покупки в игровой валюте "райские лепестки":

In [15]:
query = """
SELECT 'with 0 amount' AS line_name,
	COUNT(transaction_id) AS count_purchases, 
	SUM(amount) AS sum_amount, 
	MIN(amount) AS min_amount,
	MAX(amount) AS max_amount,
	AVG(amount::NUMERIC)::NUMERIC(10,2) AS avg_amount,
	(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY amount))::NUMERIC(10,2) AS median_amount,
	STDDEV(amount)::NUMERIC(10,2) AS stddev_amount
FROM fantasy.events
UNION
SELECT 'without 0 amount' AS line_name,
	COUNT(transaction_id) AS count_purchases, 
	SUM(amount) AS sum_amount, 
	MIN(amount) AS min_amount,
	MAX(amount) AS max_amount,
	AVG(amount::NUMERIC)::NUMERIC(10,2) AS avg_amount,
	(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY amount))::NUMERIC(10,2) AS median_amount,
	STDDEV(amount)::NUMERIC(10,2) AS stddev_amount
FROM fantasy.events
WHERE amount != 0;
"""
pd.read_sql(query, con=engine)

,line_name,count_purchases,sum_amount,min_amount,max_amount,avg_amount,median_amount,stddev_amount
0,with 0 amount,1307678,686615040.0,0.00,486615.1,525.69,74.86,2517.35
1,without 0 amount,1306771,686615040.0,0.01,486615.1,526.06,74.86,2518.18


Было 1 307 678 покупок.   
Присутствуют покупки с нулевой стоимостью.  
Самая дорогая покупка стоила - 486 615.1  
Самая дешевая исключая нулевые стоимости - 0.01  
Что значительно превышает среднюю стоимость - 525.69  
Медиана - 74.86, также сильно отличается от среднего значения, что указывает на доминирующее количество дешевых покупок и присутствие единичных очень дорогих.   
Это подтверждает стандартное отклонение - 2 517.35 , оно сильно превышает среднее значение, что говорит о большом разбросе.  

Доля покупок с нулевой стоимостью:

In [16]:
query = """
SELECT id ,
	COUNT(id) AS count_id,
	item_code , 
	game_items ,
	MIN(amount_is_zero) AS amount_is_zero ,
	MIN(share_amount_is_zero) AS share_amount_is_zero
FROM (
	SELECT item_code , amount, id ,
		SUM(CASE WHEN amount=0 THEN 1 ELSE 0 END ) OVER() AS amount_is_zero,
		(SUM(CASE WHEN amount=0 THEN 1 ELSE 0 END )OVER()/COUNT(*)OVER()::NUMERIC)::NUMERIC(10, 5)  AS share_amount_is_zero
	FROM fantasy.events
	) AS item_zero_amount
JOIN fantasy.items USING(item_code)
WHERE amount = 0
GROUP BY id , 
	  item_code , 
	  game_items 
ORDER BY count_id DESC;
"""
pd.read_sql(query, con=engine)

,id,count_id,item_code,game_items,amount_is_zero,share_amount_is_zero
0,12-1058351,810,6010,Book of Legends,907,0.00069
1,42-0460342,6,6010,Book of Legends,907,0.00069
2,06-2087517,6,6010,Book of Legends,907,0.00069
3,10-9330719,5,6010,Book of Legends,907,0.00069
4,72-8559492,3,6010,Book of Legends,907,0.00069
...,...,...,...,...,...,...
67,08-9668789,1,6010,Book of Legends,907,0.00069
68,11-3246483,1,6010,Book of Legends,907,0.00069
69,11-9176854,1,6010,Book of Legends,907,0.00069
70,15-5750613,1,6010,Book of Legends,907,0.00069


907 покупок с нулевой стоимостью. Их доля от общего количества составляет 0.00069, что меньше 1%  
Покупался за нулевую стоимость только предмет Book of Legends.  
Сильно выделился игрок 12-1058351, он купил 810 Book of Legends.  

Популярность эпических предметов - кол-во продаж, доля покупателей:

In [17]:
query = """
WITH all_buyers AS(
	SELECT id, 
		COUNT(*) OVER() AS count_all_buyers
	FROM fantasy.events
	GROUP BY id
)
SELECT item_code, 
	game_items,
	ROUND(AVG(amount)::NUMERIC,2) AS avg_amount,
	COUNT(*) AS count_purchases_item,
	COUNT(*)::NUMERIC/SUM(COUNT(*))OVER() AS share_item,
	COUNT(DISTINCT id) AS count_unique_buyers,
	COUNT(DISTINCT id)/MIN(count_all_buyers)::NUMERIC AS share_buyers
FROM fantasy.events
LEFT JOIN fantasy.items USING(item_code)
LEFT JOIN all_buyers USING(id)
WHERE amount != 0
GROUP BY item_code , game_items 
ORDER BY count_purchases_item DESC;
"""
pd.read_sql(query, con=engine)

,item_code,game_items,avg_amount,count_purchases_item,share_item,count_unique_buyers,share_buyers
0,6010,Book of Legends,435.34,1004516,7.687009e-01,12194,0.884072
1,6011,Bag of Holding,885.17,271875,2.080510e-01,11968,0.867687
2,6012,Necklace of Wisdom,402.12,13828,1.058181e-02,1627,0.117958
3,6536,Gems of Insight,383.78,3833,2.933184e-03,926,0.067136
4,5964,Treasure Map,40.83,3084,2.360016e-03,753,0.054593
...,...,...,...,...,...,...,...
140,4215,Bow of Precision,129.14,1,7.652450e-07,1,0.000073
141,8011,Incubus Whisper,481.75,1,7.652450e-07,1,0.000073
142,2741,Elixir of Mana,113.59,1,7.652450e-07,1,0.000073
143,9222,Monk's Prayer Beads,38.18,1,7.652450e-07,1,0.000073


Запрос считающий кол-во ни разу не купленных предметов и их долю от всех предметов:

In [18]:
query = """
SELECT 'count_not_bought_items' AS result,
	COUNT(*)
FROM fantasy.items
LEFT JOIN fantasy.events ON events.item_code = items.item_code WHERE events.item_code IS NULL
UNION ALL
SELECT 'count_all_items' AS result,
	COUNT(*)
FROM fantasy.items
UNION ALL
SELECT 'share_items_from_all_count' AS result,
	(SELECT COUNT(*) AS count_not_bought_items
	FROM fantasy.items
	LEFT JOIN fantasy.events ON events.item_code = items.item_code WHERE events.item_code IS NULL)
	/(SELECT COUNT(*) AS count_all_items
	FROM fantasy.items)::NUMERIC;
"""
pd.read_sql(query, con=engine)

,result,count
0,count_not_bought_items,39.000000
1,count_all_items,184.000000
2,share_items_from_all_count,0.211957


Самые продаваемые предметы:   
Book of Legends(76.87% от всех покупок, 88.4% доля купивших игроков)  
Bag of Holding(20.81% от всех покупок, 86.77% доля купивших игроков)   
Доля остальных предметов менее 2%. А доля покупателей не превышает 12%. Это указывает на явное смещение баланса. 2 самых популярных предмета выглядят как почти обязательные для геймплея  
39 предметов ни разу не были куплены. Их доля составляет 21.19% от всего магазина  

**Вывод:**  
Demon - реже всего покупают - 59.97%, но больше всех платят - 19.94%  
Northman, Elf - имеют самые высокие средние чеки - 761.5 и 682.34. Но Elf имеют самую низкую долю платящих игроков из покупающих - 16.26%  

Разброс доли покупающих и доли платящих игроков небольшой:  
59.97% - 62.89% - покупающие  
16.26% - 19.94% - платящие  
Мы видим подтверждение теории: игра за персонажей разных рас требует примерно равного количества покупок эпических предметов.  
Elf и Northman предпочитают брать в среднем меньше покупок, но дорогостоящих.  
Human покупают больше всего вещей, но средний суммарный чек на 3 месте лидеров  

_______
<a class="anchor" id="5-bullet"></a>
## Общие выводы и рекомендации:

**Общий вывод:** покупки равномерны по расам, но в дисбалансе по эпическим предметам. Доля платящих примерно одинаковая.

**Рекомендации:**  
1. Посмотреть с чем связана такая популярность Book of Legends и Bag of Holding у игроков  
2. Попробовать ввести конкурентные предметы, которые разделят популярность. Дисбаланс огромный и это мешает развитию внутриигровой экономики. Есть вероятность, что проблема не в ценах, а в ценности предметов для геймплея. Игроки готовы платить ~800 за ценный предмет.  
3. Средний чек очень низкий. Можно попробовать его увеличить. Например, новые предложения, боксы, акции  
    3.1. Самый высокий чек у Northman, Elf. Их можно мотивировать на большие покупки  
    3.2. Demon больше всего покупают валюту, это тоже можно использовать. Но их надо стимулировать на покупки в целом  
4. Убрать предметы, которые не покупали ни разу, из магазина или изменить их свойства и подать как полезную новинку(возможно сезонную)  
5. Можно разработать предложения расам в соответствии с их покупательским поведением  
    5.1. Human, Orc, Hobbit любят покупать много не сильно дорогостоящих предметов. Являются основным потоком. Подойдут дешевые акции, скидки  
    5.2. Elf, Northman покупают в среднем мало предметов, но по самому большому среднему чеку. Есть смысл продавать дорогие предметы  
    5.3. Demon покупают мало предметов, суммарный средний чек самый низкий, но самая большая доля платящих  
    5.4. Angel много мелких покупок, но средний суммарный чек не низкий. Низкая доля платящих  